# ab1scope: Interactive Analysis of Sanger Sequencing Data

Welcome to **ab1scope**, an interactive toolkit for processing `.ab1` files generated from Sanger DNA sequencing. This notebook is designed to help you explore, visualize, and curate electropherogram trace data, offering key features such as:

- Interactive selection of signal filtering and smoothing parameters  
- Real-time visualization of the four-base trace signals (A, C, G, T)  
- Generation of filtered base sequences (including IUPAC notation)  
- Quality diagnostics using intensity, dominance, and Phred-like metrics  
- Export of results in FASTA and CSV formats  
- Batch processing of multiple `.ab1` files

This tool is ideal for researchers, students, and laboratory technicians looking to:
- Evaluate the quality of Sanger sequencing runs  
- Identify ambiguities and sequencing artifacts  
- Extract reliable sequences for downstream applications  

Make sure your `.ab1` files are located in a designated folder before you begin. Each file will be analyzed independently, and all outputs (graphs, sequences, and tables) will be saved to a `/data` subfolder within the selected directory.

> **Note**: This notebook is optimized for interactive use in Jupyter environments and relies on widgets for parameter control.

---

## How to Use This Notebook

1. **Set the input folder** containing `.ab1` files  
2. **Run the interactive cell** to analyze individual files  
3. **Review the chromatogram, sequence, and diagnostics**  
4. **Export results** for each file (CSV and FASTA)  
5. Optionally, **run batch functions** to apply analysis to all files automatically

Let's get started!


In [1]:
import plotly.io as pio
import sys
import os
import numpy as np
import pandas as pd

sys.path.append(os.path.abspath(".."))
import ab1scope

## 📂 Seleção da Pasta de Entrada e Visualização dos Arquivos

Nesta etapa, você define o caminho para a pasta que contém os arquivos `.ab1` a serem analisados.

- A variável `path` deve apontar para o diretório com os arquivos de sequenciamento.
- A seguir, o notebook lista automaticamente todos os arquivos `.ab1` encontrados nessa pasta.
- Em seguida, é executada a função `process_ab1()` do `ab1scope`, que abre uma interface interativa para visualização e análise de um arquivo por vez.

> ⚙️ A interface permite ajustar parâmetros como intensidade mínima e máxima, razão de dominância entre bases, normalização e suavização dos sinais, além do zoom sobre a região de interesse.

In [2]:
pio.renderers.default = "notebook_connected"

path = r"C:\Users\borge\Documents\proc_sanger\HUGO_LIMA_02seqs_BIOF-06062025_2025-06-06-13-11-55"

# Lista apenas arquivos com extensão .ab1 (case-insensitive)
ab1_files = [f for f in os.listdir(path) if f.lower().endswith(".ab1")]

# Exibe os arquivos encontrados
print("Arquivos .ab1 encontrados:")
for f in ab1_files:
    print(f)

state = ab1scope.process_ab1(ab1_dir=path, return_data=True)

Arquivos .ab1 encontrados:
HUGO_LIMA_01_E07_2025-06-06-13-11-55_PSEQDNA.ab1
HUGO_LIMA_02_F07_2025-06-06-13-11-55_PSEQDNA.ab1


Output()

Output()

## 📊 Registro e Visualização dos Parâmetros de Processamento

Após a análise interativa de cada arquivo `.ab1`, os parâmetros utilizados (como limites de intensidade, razão de dominância, uso de normalização e suavização) são armazenados no dicionário `state`.

Esta célula extrai os parâmetros utilizados para cada arquivo analisado e organiza essas informações em um `DataFrame` (`df_params`) para documentação e inspeção.

### O que é exibido:
- Nome do arquivo `.ab1`  
- Parâmetros ajustados manualmente durante a análise interativa  
- Intervalo de zoom e configurações de pré-processamento

Essa tabela é útil para:
- Verificar quais parâmetros foram utilizados em cada análise  
- Reproduzir ou justificar decisões durante o processamento de sequências  
- Comparar a consistência entre diferentes arquivos processados


In [10]:
base_pos, A, C, G, T = state["result"]

param_rows = []

for fname, data in state.items():
    if isinstance(data, dict) and "params" in data:
        row = {"file_name": fname}
        row.update(data["params"])
        param_rows.append(row)

df_params = pd.DataFrame(param_rows)
display(df_params)


,file_name,intensity_min_threshold,intensity_max_threshold,dominance_ratio_threshold,normalize,smooth,smooth_window,zoom_range
0,HUGO_LIMA_01_E07_2025-06-06-13-11-55_PSEQDNA.ab1,0,20000,1.0,False,True,5,"(0, 1000)"
1,HUGO_LIMA_02_F07_2025-06-06-13-11-55_PSEQDNA.ab1,0,20000,1.0,False,True,5,"(0, 1000)"


## 📈 Visualização da Qualidade dos Sinais para Todos os Arquivos

Nesta etapa, aplicamos a função `plot_quality_analysis()` para **todos os arquivos `.ab1` processados anteriormente** com a função `process_ab1()`.

### O que essa célula faz:
- Percorre cada entrada no dicionário `state`, que armazena os resultados e parâmetros para cada arquivo.
- Recupera os sinais de intensidade (`A`, `C`, `G`, `T`) e as posições das bases (`base_pos`) para cada arquivo.
- Gera automaticamente o gráfico de eletroferograma com as chamadas de base, destacando as posições aceitas e rejeitadas com base nos critérios definidos.

> ✅ Útil para validar visualmente a qualidade da leitura de cada sequência processada e verificar se os parâmetros definidos produziram bons resultados.

In [11]:
# Aplicar plot_quality_analysis para todos os arquivos que foram processados
for fname, data in state.items():
    if fname == "result" or "params" not in data:
        continue  # pula a chave 'result' ou qualquer outra sem os dados esperados
    
    base_pos, A, C, G, T = data["result"]
    ab1scope.plot_quality_analysis(base_pos, A, C, G, T, file_name=fname)

✅ HTMLs saved in ./images:
- HUGO_LIMA_01_E07_2025-06-06-13-11-55_PSEQDNA.ab1_dominance_ratio.html
- HUGO_LIMA_01_E07_2025-06-06-13-11-55_PSEQDNA.ab1_total_intensity.html
- HUGO_LIMA_01_E07_2025-06-06-13-11-55_PSEQDNA.ab1_quality_score.html
✅ HTMLs saved in ./images:
- HUGO_LIMA_02_F07_2025-06-06-13-11-55_PSEQDNA.ab1_dominance_ratio.html
- HUGO_LIMA_02_F07_2025-06-06-13-11-55_PSEQDNA.ab1_total_intensity.html
- HUGO_LIMA_02_F07_2025-06-06-13-11-55_PSEQDNA.ab1_quality_score.html


## 🔬 Análise da Qualidade por Arquivo com Escore Tipo Phred

Esta célula aplica a função `analyze_quality_score_phred_like()` a todos os arquivos `.ab1` que já foram processados. A análise simula escores de qualidade no estilo **Phred**, com base na dominância do sinal da base chamada sobre os demais sinais (A, C, G, T).

### O que é feito aqui:
- Para cada arquivo no dicionário `state`, são extraídos os sinais de intensidade e as posições das bases.
- A função calcula:
  - `phred_scores`: valores simulados de qualidade para cada base chamada;
  - `dominance_ratios`: razão entre o maior e o segundo maior sinal (base dominante vs. concorrente);
  - `low_regions`: regiões da sequência com qualidade abaixo de um limiar, úteis para identificar trechos pouco confiáveis.
- Os resultados são armazenados no dicionário `quality_scores_results`, organizado por nome de arquivo.

> 🧪 Essa análise ajuda a identificar **regiões de baixa confiabilidade**, que podem ser ambíguas ou propensas a erros na chamada de bases.

In [12]:
# Agora aplicar analyze_quality_score_phred_like para todos os arquivos processados
quality_scores_results = {}

for fname, data in state.items():
    if fname == "result" or "params" not in data:
        continue  # ignora chaves que não representam arquivos analisados

    base_pos, A, C, G, T = data["result"]

    phred_scores, dominance_ratios, low_regions = ab1scope.analyze_quality_score_phred_like(
        base_positions=base_pos,
        A=A, C=C, G=G, T=T,
        file_name=fname,
        ab1_dir=path)

    quality_scores_results[fname] = {
        "phred_scores": phred_scores,
        "dominance_ratios": dominance_ratios,
        "low_regions": low_regions}

✅ Quality analysis plot saved to: C:\Users\borge\Documents\proc_sanger\HUGO_LIMA_02seqs_BIOF-06062025_2025-06-06-13-11-55\images\HUGO_LIMA_01_E07_2025-06-06-13-11-55_PSEQDNA_quality.html
✅ Quality analysis plot saved to: C:\Users\borge\Documents\proc_sanger\HUGO_LIMA_02seqs_BIOF-06062025_2025-06-06-13-11-55\images\HUGO_LIMA_02_F07_2025-06-06-13-11-55_PSEQDNA_quality.html


## 🧬 Detecção de Ambiguidades e Indels nas Sequências Processadas

Nesta célula, aplicamos a função `detect_ambiguities_and_indels()` a cada sequência processada. O objetivo é identificar:

- **Bases ambíguas** (representadas por códigos IUPAC como R, Y, S, etc.), que indicam presença de sinais múltiplos próximos;
- **Possíveis inserções ou deleções (indels)**, que podem ser inferidas por padrões incomuns de sinal ou regiões de baixa qualidade.

### Etapas realizadas:
- Para cada arquivo com dados válidos:
  - Recupera-se a sequência filtrada (`filtered_seq`) e os sinais de intensidade das quatro bases.
  - A função analisa cada posição da sequência para identificar potenciais **ambiguidades ou indels**.
- Os resultados são salvos no dicionário `indel_results`, com um `DataFrame` por arquivo contendo:
  - Posição
  - Base dominante
  - Bases alternativas
  - Razões de dominância
  - Classificação como ambiguidade ou potencial indel

> 🧠 Esta etapa é crucial para validar a qualidade da sequência e identificar posições problemáticas que merecem atenção ou revisão manual.


In [13]:
# Agora aplicar detect_ambiguities_and_indels para cada entrada com dados no state
indel_results = {}

for fname, data in state.items():
    if fname == "result" or "params" not in data:
        continue  # pular entradas inválidas

    base_pos, A, C, G, T = data["result"]
    iupac_seq = data["filtered_seq"]

    df_ambigs = ab1scope.detect_ambiguities_and_indels(
        base_positions=base_pos,
        A=A, C=C, G=C, T=T,
        iupac_seq=iupac_seq,
        file_name=fname,
        ab1_dir=path
    )

    # Armazena o DataFrame de ambiguidades para esse arquivo
    indel_results[fname] = df_ambigs

✅ Ambiguities and indels exported to: C:\Users\borge\Documents\proc_sanger\HUGO_LIMA_02seqs_BIOF-06062025_2025-06-06-13-11-55\data\HUGO_LIMA_01_E07_2025-06-06-13-11-55_PSEQDNA_ambiguities_and_indels.csv
✅ Ambiguities and indels exported to: C:\Users\borge\Documents\proc_sanger\HUGO_LIMA_02seqs_BIOF-06062025_2025-06-06-13-11-55\data\HUGO_LIMA_02_F07_2025-06-06-13-11-55_PSEQDNA_ambiguities_and_indels.csv


## 📊 Comparação Visual entre Arquivos .ab1

Nesta etapa, utilizamos a função `compare_ab1_samples()` para realizar uma **comparação direta** entre os arquivos .ab1 selecionados.

### O que esta função faz:
- **Carrega e processa** os sinais de intensidade (A, C, G, T) de todos os arquivos listados;
- Gera **gráficos interativos** sobrepostos que permitem:
  - Visualizar o perfil eletroforético de diferentes amostras;
  - Verificar a consistência dos picos e possíveis falhas;
  - Comparar a intensidade de sinais entre arquivos (por base e posição);
  - Detectar variações que podem indicar erros, contaminações ou diferenças biológicas.

### Aplicações:
- Verificar se dois arquivos são replicatas confiáveis;
- Avaliar se há diferenças significativas entre amostras;
- Identificar ruídos ou regiões truncadas.

> 📌 Útil especialmente para checagem de qualidade global e interpretação visual das curvas de fluorescência ao longo da leitura.


In [17]:
ab1scope.compare_ab1_samples(
    ab1_dir=path, 
    file_list=ab1_files)


First 20 polymorphic positions:


,HUGO_LIMA_01_E07_2025-06-06-13-11-55_PSEQDNA.ab1,HUGO_LIMA_02_F07_2025-06-06-13-11-55_PSEQDNA.ab1,Polymorphism
Base_Position,,,
663,T,C,True
7760,C,G,True
7895,C,A,True
7919,G,T,True
7981,T,A,True
7994,G,T,True
8019,C,A,True
8070,C,A,True
9109,T,C,True


✅ Comparison plot saved as HTML.
✅ Alignment saved to 'data/sequence_alignment.csv'


## 🔤 Geração e Exportação de Sequências com Código IUPAC

Nesta etapa, geramos as sequências de DNA utilizando **códigos IUPAC**, que permitem representar bases ambíguas de forma padronizada.

### O que está sendo feito:
- Para cada arquivo `.ab1` processado:
  - Utilizamos as intensidades de sinal das quatro bases (A, C, G, T) para **determinar a base mais provável ou combinação ambígua**, de acordo com as regras do código IUPAC;
  - A sequência gerada é então salva no formato **FASTA**, amplamente utilizado para análises bioinformáticas;
  - A sequência IUPAC também é armazenada na estrutura `state` para uso posterior no notebook.

### Exemplo de codificação IUPAC:
| Código | Significado | Bases possíveis |
|--------|-------------|------------------|
| A      | Adenina     | A                |
| R      | Purina      | A ou G           |
| Y      | Pirimidina  | C ou T           |
| N      | Qualquer    | A, C, G ou T     |

### Por que usar IUPAC?
- Permite representar **incertezas** nos picos sem perder a informação;
- É ideal para sequências que serão comparadas, alinhadas ou anotadas;
- Facilita o rastreio de regiões problemáticas em análises futuras.

> 📁 Os arquivos FASTA são salvos automaticamente no diretório `data/`, dentro da pasta onde estão os arquivos `.ab1`.


In [15]:
# gerar sequência com IUPAC:
for fname, data in state.items():
    if fname == "result" or "params" not in data:
        continue  # ignora chaves inválidas

    base_pos, A, C, G, T = data["result"]

    # Gera a sequência com códigos IUPAC
    iupac_seq = ab1scope.call_iupac_sequence(base_pos, A, C, G, T)

    # Salva em formato FASTA
    ab1scope.export_iupac_fasta(iupac_seq, fname, path)

    # Opcional: salva também no próprio state
    state[fname]["iupac_seq"] = iupac_seq

✅ IUPAC FASTA saved: C:\Users\borge\Documents\proc_sanger\HUGO_LIMA_02seqs_BIOF-06062025_2025-06-06-13-11-55\data\HUGO_LIMA_01_E07_2025-06-06-13-11-55_PSEQDNA_iupac.fasta
✅ IUPAC FASTA saved: C:\Users\borge\Documents\proc_sanger\HUGO_LIMA_02seqs_BIOF-06062025_2025-06-06-13-11-55\data\HUGO_LIMA_02_F07_2025-06-06-13-11-55_PSEQDNA_iupac.fasta


## ⚙️ Geração e Exportação Automatizada de Sequências IUPAC

Esta célula executa de forma **automatizada** a geração das sequências em código IUPAC para **todos os arquivos `.ab1`** listados, com base nos resultados previamente processados e otimizados armazenados no dicionário `state`.

### O que faz a função:
- Itera sobre cada arquivo `.ab1` listado em `ab1_files`;
- Utiliza as intensidades de sinal (A, C, G, T) e posições já processadas;
- Aplica as regras do código IUPAC para gerar a sequência correspondente;
- Exporta a sequência no formato **FASTA** para a pasta `data/`;
- Atualiza o dicionário `state` com as sequências IUPAC geradas.

### Vantagens:
- **Evita retrabalho**, reaproveitando os parâmetros ajustados via interface interativa;
- Garante consistência e reprodutibilidade dos resultados;
- Facilita a documentação e o compartilhamento das sequências.

> 💾 Os arquivos `.fasta` gerados podem ser utilizados diretamente em pipelines de alinhamento, BLAST ou montagem de contigs.


In [18]:
# Executar geração e exportação para todos os arquivos
ab1scope.generate_and_export_iupac_for_files(ab1_files, path, state=state)

Processando: HUGO_LIMA_01_E07_2025-06-06-13-11-55_PSEQDNA.ab1
✅ IUPAC FASTA saved: C:\Users\borge\Documents\proc_sanger\HUGO_LIMA_02seqs_BIOF-06062025_2025-06-06-13-11-55\data\HUGO_LIMA_01_E07_2025-06-06-13-11-55_PSEQDNA_iupac.fasta
Processando: HUGO_LIMA_02_F07_2025-06-06-13-11-55_PSEQDNA.ab1
✅ IUPAC FASTA saved: C:\Users\borge\Documents\proc_sanger\HUGO_LIMA_02seqs_BIOF-06062025_2025-06-06-13-11-55\data\HUGO_LIMA_02_F07_2025-06-06-13-11-55_PSEQDNA_iupac.fasta
✅ Todas as sequências foram exportadas para FASTA com código IUPAC.


{'result': (array([    3,    15,    29, ..., 15002, 15023, 15058], shape=(1129,)),
  array([ 72. ,  97.2, 123.2, ...,  52.6,  42.6,  32.2], shape=(15071,)),
  array([19.4, 28. , 37.4, ..., 21.2, 18.6, 14.8], shape=(15071,)),
  array([ 32. ,  42.8,  54. , ..., 272. , 218.4, 164.2], shape=(15071,)),
  array([10.4, 15.8, 22.2, ..., 35.6, 28.4, 21.4], shape=(15071,))),
 'filtered_seq': 'AAAAAAACAATGGGCCTTCGATCCAGTACCTACCTTGGGTAAAAGACAGAATAACTAAAGACAGCCTATAGACAACGTGATTTGGCAATAAAAACTCCTGGGGAACCCTAACCATGCCGCTTTTCGCGAATACACATGACCCATGGCCGAGATTGACAACGACCTGTATAGTGACGGCATGCAGGAAGAAAAGCCACATGGCTTCCAATACTACGTCAGCATGAGGTGACACAACCACGTCATTCTCTGAAACTGACACTCTAATTTCACTGTTGCTAAACAATCATCAGAAAACCGGGGTTATCCGAAATATCCATTGACTAACATCCTATTATAGACAAACAACCGAGTGAGAACGGAGCCCTTGTGGGGTGTGAAACCACCACGTGACTAAGCTTCACAACCGTCATCGTAATGCCTGTTAGATTCATCATTATAAGCCACTAACAACGATCCATAGGCGTTAACCGGCGGAAATAGCCCATAGTTAGCAATAAGTCCGGCCAGTCAGGAGGCCCACTGATCATGTCCTGAGAAACACATCCTGACCCTGACGGCCTGCACAGCGAAGCAAAAGCACAAACCGGTTGCAGAGCATAAGACCCGATAGCACG